In [2]:
import pandas as pd
import os 
import re
import nltk
from nltk.tokenize import word_tokenize
import boto3
from langdetect import detect
from collections import Counter

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
news = pd.read_excel('news_excerpts_parsed.xlsx')
leaks = pd.read_excel('wikileaks_parsed.xlsx')

In [5]:
leaks['Text'] = leaks['Text'].astype(str)
news['Text'] = news['Text'].astype(str)

In [6]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1509 entries, 0 to 1508
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Link    1509 non-null   object
 1   Text    1509 non-null   object
dtypes: object(2)
memory usage: 23.7+ KB


In [7]:
leaks

,PDF Path,Text
0,1.pdf,Pristina Airport – Possible administrative irr...
1,1.pdf,Investigative details\n\nIn his/her interviews...
2,10.pdf,"""An interoffice memorandum providing an “outst..."
3,10.pdf,"""Allegation 2 & 3:\n\n(Specifically, three of ..."
4,10.pdf,"""When asked about this in interview, the Divis..."
...,...,...
138,89.pdf,"""Description\n\nTop Secret US National Securit..."
139,9.pdf,"""INTRODUCTION\n\nThis case arises out of an au..."
140,9.pdf,"""BACKGROUND INFORMATION\n\nPristina Internatio..."
141,9.pdf,"""BACKGROUND INFORMATION\n\nPristina Internatio..."


In [8]:
leaks['Text'][1]

'Investigative details\n\nIn his/her interviews conducted on 31st August and 14th September 2004, Vendor 1 and Vendor 2 Representative admitted that the fact that both Vendor 1 and Vendor 2 took part together in three Airport tenders put other competitors at a disadvantage, but alleged both companies have never exchanged information with regard to the price offers.\n\nThere were doubts whether both companies Vendor 3 and Vendor 4 exist and submitted bids for the three tenders. The ITF investigation, with support from the Kosovo Organised Crime Bureau (KOCB), has found no reference with regard to the existence of the alleged competitor Vendor 4 in Pristina. The alleged company Vendor 3 was found to be a supermarket located in Prizren.\n\nThe above-mentioned facts are already part of the ITF case no. 286/04 that was submitted to the International Prosecutor, UNMIK Department of Justice, in October 2004. On 1 December 2004, the ITF contacted the International Prosecutor for a legal assess

In [9]:
def check_for_patterns(df, patterns):
    results = {}
    for pattern in patterns:
        contains_pattern = df['Text'].str.contains(pattern, na=False)
        results[pattern] = contains_pattern.sum()  # Count rows containing the pattern
    return results

# Patterns to check for
patterns_to_check = ['\n', '\xad', '\t', '�', r'\\']

# Check both datasets for the patterns
print(f"News : {check_for_patterns(news, patterns_to_check)}")
print(f"Leaks : {check_for_patterns(leaks, patterns_to_check)}")

News : {'\n': 1497, '\xad': 0, '\t': 0, '�': 0, '\\\\': 0}
Leaks : {'\n': 93, '\xad': 5, '\t': 0, '�': 0, '\\\\': 1}


In [10]:
leaks['Text'] = leaks['Text'].str.replace('\n', '.', regex=True)
news['Text'] = news['Text'].str.replace('\n', '.', regex=True)
leaks['Text'] = leaks['Text'].str.replace('\xad', '-', regex=True)
leaks['Text'] = leaks['Text'].str.replace( r'\\', "''", regex=True)

In [11]:
leaks

,PDF Path,Text
0,1.pdf,Pristina Airport – Possible administrative irr...
1,1.pdf,Investigative details..In his/her interviews c...
2,10.pdf,"""An interoffice memorandum providing an “outst..."
3,10.pdf,"""Allegation 2 & 3:..(Specifically, three of th..."
4,10.pdf,"""When asked about this in interview, the Divis..."
...,...,...
138,89.pdf,"""Description..Top Secret US National Security ..."
139,9.pdf,"""INTRODUCTION..This case arises out of an audi..."
140,9.pdf,"""BACKGROUND INFORMATION..Pristina Internationa..."
141,9.pdf,"""BACKGROUND INFORMATION..Pristina Internationa..."


In [12]:
leaks['Text'][52]

'With regards to performance and conduct of the employees of KTA, the following UNMIK Regulation shall apply: ..UNMIK Regulation No. 2001/27, Dated 8 October 2001- On Essential Labour Law In Kosovo - Section Termination of a Labour Contract, states the following (extract): ..Serious misconduct shall include the following: ..(a) unjustified refusal to perform the obligations set out in the labour contract; .(b) theft, destruction, damage or unauthorized use of the employer’s assets; .(c) disclosure of business secrets; .(d) consumption of drugs or alcohol at work; and .(e) behavior of such a serious nature that it would be unreasonable to expect the employment relationship to continue. ..Unsatisfactory performance shall include the following: ..(a) unjustified absence from work; and .(b) repeated mistakes not sufficient in themselves to justify a dismissal, but which given their frequency and seriousness disrupt the normal course of the employment relationship. '

In [13]:
leaks = leaks.groupby('PDF Path', as_index=False).agg({
    'Text': ' '.join  
})

In [14]:
leaks

,PDF Path,Text
0,1.pdf,Pristina Airport – Possible administrative irr...
1,10.pdf,"""An interoffice memorandum providing an “outst..."
2,105.pdf,"""Description..This is a Secret US National Sec..."
3,106.pdf,"""Tokyo's Climate Change Officials to Continue ..."
4,107.pdf,"""Description..This is a secret US National Sec..."
5,108.pdf,"""Japanese Strive to Avoid Damage to U.S. Relat..."
6,11.pdf,"Until the end of June 2002, responsibility for..."
7,110.pdf,The EU and Japan were engaged as of early Dece...
8,111.pdf,A 22 October meeting attended by German Chance...
9,112.pdf,"Israel has reached out to Europe, including It..."


In [15]:
leaks['Text'][0]

'Pristina Airport – Possible administrative irregularity regarding tender procedures involving Vendor 1 and Vendor 2..Allegation..Two companies with the same owner took part at least three times in the same Airport tenders...Background Information..The Kosovo citizen, Vendor 1 and Vendor 2 Representative, is the owner and Director of the Pristina-based Vendor 1 and also a 51% shareholder of the Pristina-Ljubljana-based company Vendor 2. Both companies have their residences at the same address in Pristina...Both Vendor 1 and Vendor 2 submitted three times in 2003 for the same tenders:..Supply and Mounting of Sonic System in the Fire Station Building. Winner was Vendor 2 with €1,530 followed by Vendor 1 with €1,620. The third company, Vendor 3, did not provide a price offer...Cabling of Flat Display Information System (FIDS). Winner was Vendor 1 with €15,919 followed by Vendor 2 with €19,248.70. The other two competitors, Vendor 3 and Vendor 4, offered prices of Euro 19,702 and Euro 21,0

In [16]:
leaks['Token_Count'] = leaks['Text'].apply(lambda x: len(word_tokenize(x)))

In [17]:
news['Token_Count'] = news['Text'].apply(lambda x: len(word_tokenize(x)))

In [18]:
leaks

,PDF Path,Text,Token_Count
0,1.pdf,Pristina Airport – Possible administrative irr...,426
1,10.pdf,"""An interoffice memorandum providing an “outst...",1879
2,105.pdf,"""Description..This is a Secret US National Sec...",378
3,106.pdf,"""Tokyo's Climate Change Officials to Continue ...",312
4,107.pdf,"""Description..This is a secret US National Sec...",256
5,108.pdf,"""Japanese Strive to Avoid Damage to U.S. Relat...",343
6,11.pdf,"Until the end of June 2002, responsibility for...",475
7,110.pdf,The EU and Japan were engaged as of early Dece...,319
8,111.pdf,A 22 October meeting attended by German Chance...,232
9,112.pdf,"Israel has reached out to Europe, including It...",216


In [19]:
news

,Link,Text,Token_Count
0,https://edition.cnn.com/2023/09/29/business/st...,Starbucks violated federal labor law when it i...,127
1,https://www.channelnewsasia.com/singapore/su-w...,The first suspect to plead guilty in Singapore...,145
2,https://edition.cnn.com/2023/05/22/tech/meta-f...,Meta has been fined a record-breaking €1.2 bil...,153
3,https://www.channelnewsasia.com/singapore/bill...,SINGAPORE: A 45-year-old man linked to Singapo...,116
4,https://edition.cnn.com/2024/03/05/politics/li...,The Department of Education imposed a record $...,146
...,...,...,...
1504,https://www.channelnewsasia.com/commentary/mal...,NOT A SURPRISE FOR INDUSTRY FOLLOWERS..It is h...,136
1505,https://www.channelnewsasia.com/singapore/tick...,"Ahead of the Tokyo 2020 Olympics, Japan prohib...",144
1506,https://www.bbc.com/news/world-europe-57965260,At a court hearing on Monday in the city of Ki...,147
1507,https://www.bbc.com/news/uk-wales-mid-wales-13...,"Ray Mann, managing director of the West Wales ...",120


In [20]:
print('inv' if any(news['Token_Count'] > 300) else 'ok')
print('inv' if any(leaks['Token_Count'] > 4000) else 'ok')

ok
ok


In [21]:
language_counts_news = Counter(news['Text'].apply(detect))
language_counts_leaks = Counter(leaks['Text'].apply(detect))
print(language_counts_news)
print(language_counts_leaks)

Counter({'en': 1509})
Counter({'en': 44})


In [22]:
news

,Link,Text,Token_Count
0,https://edition.cnn.com/2023/09/29/business/st...,Starbucks violated federal labor law when it i...,127
1,https://www.channelnewsasia.com/singapore/su-w...,The first suspect to plead guilty in Singapore...,145
2,https://edition.cnn.com/2023/05/22/tech/meta-f...,Meta has been fined a record-breaking €1.2 bil...,153
3,https://www.channelnewsasia.com/singapore/bill...,SINGAPORE: A 45-year-old man linked to Singapo...,116
4,https://edition.cnn.com/2024/03/05/politics/li...,The Department of Education imposed a record $...,146
...,...,...,...
1504,https://www.channelnewsasia.com/commentary/mal...,NOT A SURPRISE FOR INDUSTRY FOLLOWERS..It is h...,136
1505,https://www.channelnewsasia.com/singapore/tick...,"Ahead of the Tokyo 2020 Olympics, Japan prohib...",144
1506,https://www.bbc.com/news/world-europe-57965260,At a court hearing on Monday in the city of Ki...,147
1507,https://www.bbc.com/news/uk-wales-mid-wales-13...,"Ray Mann, managing director of the West Wales ...",120


In [ ]:
news.drop(columns=['Link','Token_Count'], inplace= True)

In [27]:
leaks.drop(columns=['PDF Path','Token_Count'], inplace= True)

In [70]:
news.to_csv('news.csv', index=False)
leaks.to_csv('leaks.csv',index=False)

In [ ]:
"""
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  # For newer versions
import pyLDAvis.gensim_models
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Transform text into a document-term matrix
vectorizer = CountVectorizer(stop_words='english', max_features=5000)
doc_term_matrix = vectorizer.fit_transform(news['Text'])

# Fit an LDA model
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(doc_term_matrix)

# Visualization for pyLDAvis
pyLDAvis.enable_notebook()

# Prepare the visualization
lda_vis = pyLDAvis.prepare(
    topic_term_dists=lda_model.components_,
    doc_topic_dists=lda_model.transform(doc_term_matrix),
    doc_lengths=[len(doc.split()) for doc in news['Text']],
    vocab=vectorizer.get_feature_names_out(),
    term_frequency=doc_term_matrix.sum(axis=0).A1
)

pyLDAvis.display(lda_vis)
"""

In [ ]:
"""
# Create a document-term matrix for the 'leaks' dataset
leaks_term_matrix = vectorizer.fit_transform(leaks['Text'])

# Fit an LDA model using the 'leaks' dataset
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(leaks_term_matrix)

# Visualization for pyLDAvis
pyLDAvis.enable_notebook()

# Prepare the visualization
lda_vis = pyLDAvis.prepare(
    topic_term_dists=lda_model.components_,
    doc_topic_dists=lda_model.transform(leaks_term_matrix),
    doc_lengths=[len(doc.split()) for doc in leaks['Text']],
    vocab=vectorizer.get_feature_names_out(),
    term_frequency=leaks_term_matrix.sum(axis=0).A1
)

pyLDAvis.display(lda_vis)
"""

In [51]:
def upload_files_to_s3(bucket_name, folder_path='.'):
    # Create an S3 client
    s3_client = boto3.client('s3')
    
    # Get all files from the root folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    for file in files:
        # The S3 key will be the file name as-is
        s3_key = file
        
        # Upload file to S3
        print(f"Uploading {file} to s3://{bucket_name}/{s3_key}")
        s3_client.upload_file(file, bucket_name, s3_key)

    print("All files uploaded successfully!")

In [ ]:
upload_files_to_s3('bia-datathon-text-data-unstructured', folder_path)

Uploading leaks.csv to s3://bia-datathon-text-data-unstructured/leaks.csv
